In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

In [4]:
import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
import evosax

import json
import math
import drawsvg

In [5]:
import xylo
import xylo.types as t
import xylo.woods
import xylo.cut

import xylo.opt

import xylo.tuning
import xylo.database

import xylo.just

import xylo.resonators as r

In [6]:
layout = xylo.just.bass_partch

In [7]:
c = r.Config(radius = 0.090/2)
c

Config(temp=20, correction=0.61, radius=0.045)

In [8]:
slack = 0.05
tube_lengths = { k: r.length_of_freq(f, c) + slack for k,f in layout.get_freqs().items() }

In [9]:
sum([v for v in tube_lengths.values() if v > 0.00])

8.304666333352966

In [10]:
tube_lengths

{'A#3': 0.39778464578997946,
 'A3': 0.41168222526368253,
 'F#3': 0.5001213673690649,
 'F3': 0.5150454725993482,
 'D#3': 0.5697671917770535,
 'C#3': 0.6244889109547588,
 'A2': 0.7886540684878749,
 'G2': 0.8980975068432856,
 'E2': 1.0732070082119427,
 'D2': 1.1899466757910475,
 'C2': 1.3358712602649285}

In [11]:
for k, v in tube_lengths.items():
  if v >= 0.08:
    print(f"{k}: {v}m ({layout.get_freqs()[k]}Hz)")

A#3: 0.39778464578997946m (228.66666666666669Hz)
A3: 0.41168222526368253m (220.5Hz)
F#3: 0.5001213673690649m (179.66666666666666Hz)
F3: 0.5150454725993482m (174.2222222222222Hz)
D#3: 0.5697671917770535m (156.8Hz)
C#3: 0.6244889109547588m (142.54545454545456Hz)
A2: 0.7886540684878749m (112.0Hz)
G2: 0.8980975068432856m (98.0Hz)
E2: 1.0732070082119427m (81.66666666666667Hz)
D2: 1.1899466757910475m (73.5Hz)
C2: 1.3358712602649285m (65.33333333333333Hz)


In [12]:
def mkres(lens, filename, offsetx = 1.3):
  d = drawsvg.Drawing('297mm', '210mm')
  d.append(drawsvg.Rectangle(0, 0, '297mm', '210mm', stroke='green', stroke_width=2, fill='none'))
  d.append(drawsvg.Text(filename, '5mm', '20mm', '8mm'))
  grp = drawsvg.Group()
  y = 10
  for k,v in lens.items():
    mm = v * 1000 + offsetx
    print(k,v)
    grp.append(drawsvg.Text(f'{k}', '2mm', f'{mm-1}mm', f'{y + 3}mm'))
    grp.append(drawsvg.Circle(f'{mm}mm', f'{y + 1}mm', '0.5mm'))
    grp.append(drawsvg.Rectangle(f'{offsetx}mm', f'{y}mm', f'{mm}mm', f'0.5mm', stroke='red', stroke_width=1))
  grp.append(drawsvg.Circle(f'{offsetx}mm', f'{y+1}mm', '1mm'))

  d.append(grp)

  d.save_html(f'data/layout/{filename}.html')
  return d

mkres(tube_lengths, 'dia-resonator-lengths')

A#3 0.39778464578997946
A3 0.41168222526368253
F#3 0.5001213673690649
F3 0.5150454725993482
D#3 0.5697671917770535
C#3 0.6244889109547588
A2 0.7886540684878749
G2 0.8980975068432856
E2 1.0732070082119427
D2 1.1899466757910475
C2 1.3358712602649285


FileNotFoundError: [Errno 2] No such file or directory: 'data/layout/dia-resonator-lengths.html'